# 📄 Document Classification Pipeline (Clean Version)

## EDA 기반 최적화된 문서 분류 파이프라인

### Contents
1. **Setup & Configuration** - 라이브러리, 설정, 하이퍼파라미터
2. **Data Processing** - 데이터셋, 전처리, 증강
3. **Model Training** - 모델 학습 및 검증
4. **Inference & Submission** - 추론 및 결과 저장

### Key Features
- ✅ EDA 기반 클래스 불균형 해결
- ✅ Train/Test 도메인 차이 대응
- ✅ 클래스별 맞춤 전처리
- ✅ 오버피팅 방지 (Train/Val 분할)
- ✅ Test Time Augmentation


## 1. Setup & Configuration


In [1]:
# Core Libraries
import os
import random
import datetime
import warnings
warnings.filterwarnings('ignore')

# Deep Learning
import torch
import torch.nn as nn
import timm
from torch.utils.data import Dataset, DataLoader

# Data Processing
import pandas as pd
import numpy as np
import cv2
from PIL import Image
import albumentations as A
from albumentations.pytorch import ToTensorV2

# ML Utils
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from tqdm import tqdm

# Experiment Tracking
import wandb

print("✅ Libraries imported successfully!")


✅ Libraries imported successfully!


In [2]:
# Configuration Class
class Config:
    # Paths
    DATA_PATH = "/home/dev/computervisioncompetition-cv3/data"
    
    # Model Settings
    MODEL_NAME = 'efficientnet_b0'
    IMG_SIZE = 512
    NUM_CLASSES = 17
    
    # Training Settings
    BATCH_SIZE = 16
    EPOCHS = 50
    LEARNING_RATE = 1e-4
    WEIGHT_DECAY = 1e-2
    
    # Data Settings
    VAL_RATIO = 0.2
    NUM_WORKERS = 4
    
    # EDA-based Settings
    USE_CLASS_WEIGHTS = True
    USE_CLASS_SPECIFIC_AUG = True
    EARLY_STOPPING_PATIENCE = 5
    
    # Random Seed
    SEED = 42
    
    # Device
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# EDA-based Class Information
CLASS_WEIGHTS = {
    0: 0.92, 1: 2.01, 2: 0.92, 3: 0.92, 4: 0.92, 5: 0.92, 6: 0.92, 7: 0.92, 8: 0.92,
    9: 0.92, 10: 0.92, 11: 0.92, 12: 0.92, 13: 1.25, 14: 1.85, 15: 0.92, 16: 0.92
}

VEHICLE_CLASSES = [2, 16]  # car_dashboard, vehicle_registration_plate
MINORITY_CLASSES = [1, 13, 14]  # Underrepresented classes

def set_seed(seed):
    """Fix random seeds for reproducibility"""
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = True

def get_class_weights_tensor():
    """Convert class weights to tensor"""
    weights = [CLASS_WEIGHTS[i] for i in range(Config.NUM_CLASSES)]
    return torch.FloatTensor(weights)

# Initialize
set_seed(Config.SEED)
print(f"🚀 Configuration loaded - Device: {Config.DEVICE}")
print(f"   - Model: {Config.MODEL_NAME}")
print(f"   - Image Size: {Config.IMG_SIZE}x{Config.IMG_SIZE}")
print(f"   - Batch Size: {Config.BATCH_SIZE}")
print(f"   - Learning Rate: {Config.LEARNING_RATE}")
print(f"   - Use Class Weights: {Config.USE_CLASS_WEIGHTS}")


🚀 Configuration loaded - Device: cuda
   - Model: efficientnet_b0
   - Image Size: 512x512
   - Batch Size: 16
   - Learning Rate: 0.0001
   - Use Class Weights: True


## 2. Data Processing


In [3]:
# Data Transforms
def get_transforms(mode='train', image_size=512):
    """Get data transforms based on mode"""
    
    if mode == 'train':
        return A.Compose([
            # Basic preprocessing
            A.LongestMaxSize(max_size=image_size, interpolation=cv2.INTER_AREA),
            A.PadIfNeeded(image_size, image_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            
            # EDA-based augmentations
            A.RandomRotate90(p=0.3),
            A.Rotate(limit=30, p=0.7, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.8),
            
            # Noise and blur (Test data characteristics)
            A.OneOf([
                A.GaussNoise(var_limit=(10, 50)),
                A.MultiplicativeNoise(multiplier=[0.9, 1.1], elementwise=True),
            ], p=0.5),
            A.OneOf([
                A.MotionBlur(blur_limit=3),
                A.GaussianBlur(blur_limit=3),
            ], p=0.3),
            
            # Geometric transforms
            A.Perspective(scale=(0.05, 0.1), p=0.3),
            A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=0, p=0.3,
                             border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            
            # Document-specific augmentations
            A.CoarseDropout(max_holes=1, max_height=32, max_width=32, fill_value=255, p=0.3),
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.5),
            
            # Normalization
            A.Normalize(mean=[0.57, 0.58, 0.59], std=[0.24, 0.24, 0.24]),
            ToTensorV2(),
        ])
    
    else:  # validation/test
        return A.Compose([
            A.LongestMaxSize(max_size=image_size, interpolation=cv2.INTER_AREA),
            A.PadIfNeeded(image_size, image_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=0.3),
            A.Normalize(mean=[0.57, 0.58, 0.59], std=[0.24, 0.24, 0.24]),
            ToTensorV2(),
        ])

def get_tta_transforms(image_size=512):
    """Get Test Time Augmentation transforms"""
    transforms = []
    
    # Base transform
    transforms.append(get_transforms('test', image_size))
    
    # Horizontal flip
    transforms.append(A.Compose([
        A.LongestMaxSize(max_size=image_size, interpolation=cv2.INTER_AREA),
        A.PadIfNeeded(image_size, image_size, border_mode=cv2.BORDER_CONSTANT, value=[255, 255, 255]),
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=[0.57, 0.58, 0.59], std=[0.24, 0.24, 0.24]),
        ToTensorV2()
    ]))
    
    return transforms

print("✅ Data transforms defined")


✅ Data transforms defined


In [4]:
# Dataset Class
class DocumentDataset(Dataset):
    """Enhanced dataset with class-specific preprocessing"""
    
    def __init__(self, df, image_dir, transform=None, is_train=True):
        if isinstance(df, str):
            self.df = pd.read_csv(df)
        else:
            self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.transform = transform
        self.is_train = is_train
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        name = row['ID'] if 'ID' in row else row.iloc[0]
        target = row['target'] if 'target' in row else row.iloc[1]
        
        # Load and convert to RGB
        img_path = os.path.join(self.image_dir, name)
        img = cv2.imread(img_path)
        if img is None:
            img = np.array(Image.open(img_path))
        else:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Class-specific preprocessing (only for training)
        if self.is_train and Config.USE_CLASS_SPECIFIC_AUG:
            if target in VEHICLE_CLASSES:
                img = cv2.convertScaleAbs(img, alpha=1.1, beta=10)
            elif target in MINORITY_CLASSES:
                img = cv2.bilateralFilter(img, 9, 75, 75)
        
        # Apply transforms
        if self.transform:
            img = self.transform(image=img)['image']
            
        return img, target

print("✅ Dataset class defined")


✅ Dataset class defined


In [5]:
# Data Loading
def create_dataloaders(config):
    """Create train, validation, and test dataloaders"""
    # Load and split data
    full_df = pd.read_csv(os.path.join(config.DATA_PATH, "train.csv"))
    train_df, val_df = train_test_split(
        full_df, test_size=config.VAL_RATIO, 
        stratify=full_df['target'], random_state=config.SEED
    )
    
    # Create datasets
    train_dataset = DocumentDataset(
        train_df, os.path.join(config.DATA_PATH, "train"),
        get_transforms('train', config.IMG_SIZE), is_train=True
    )
    val_dataset = DocumentDataset(
        val_df, os.path.join(config.DATA_PATH, "train"),
        get_transforms('val', config.IMG_SIZE), is_train=False
    )
    test_dataset = DocumentDataset(
        os.path.join(config.DATA_PATH, "sample_submission.csv"),
        os.path.join(config.DATA_PATH, "test"),
        get_transforms('test', config.IMG_SIZE), is_train=False
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset, batch_size=config.BATCH_SIZE, shuffle=True,
        num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=config.BATCH_SIZE, shuffle=False,
        num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False
    )
    test_loader = DataLoader(
        test_dataset, batch_size=config.BATCH_SIZE, shuffle=False,
        num_workers=config.NUM_WORKERS, pin_memory=True, drop_last=False
    )
    
    return train_loader, val_loader, test_loader, train_df, val_df

# Create data loaders
train_loader, val_loader, test_loader, train_df, val_df = create_dataloaders(Config)

print(f"📊 Data loaded:")
print(f"   - Train: {len(train_df):,} samples ({len(train_loader)} batches)")
print(f"   - Val: {len(val_df):,} samples ({len(val_loader)} batches)")
print(f"   - Test: {len(test_loader.dataset):,} samples ({len(test_loader)} batches)")

# Sample batch verification
sample_batch = next(iter(train_loader))
print(f"\n🔍 Sample batch:")
print(f"   - Image shape: {sample_batch[0].shape}")
print(f"   - Image range: [{sample_batch[0].min():.3f}, {sample_batch[0].max():.3f}]")
print(f"   - Label shape: {sample_batch[1].shape}")


📊 Data loaded:
   - Train: 1,256 samples (78 batches)
   - Val: 314 samples (20 batches)
   - Test: 3,140 samples (197 batches)

🔍 Sample batch:
   - Image shape: torch.Size([16, 3, 512, 512])
   - Image range: [-2.458, 1.792]
   - Label shape: torch.Size([16])


## 3. Model Training


In [6]:
# Model Components
def create_model(config):
    """Create and configure model"""
    model = timm.create_model(
        config.MODEL_NAME,
        pretrained=True,
        num_classes=config.NUM_CLASSES,
        drop_rate=0.3
    ).to(config.DEVICE)
    return model

def create_loss_function(config):
    """Create loss function with optional class weights"""
    if config.USE_CLASS_WEIGHTS:
        class_weights = get_class_weights_tensor().to(config.DEVICE)
        loss_fn = nn.CrossEntropyLoss(weight=class_weights, label_smoothing=0.1)
        print("✅ Using weighted CrossEntropyLoss")
    else:
        loss_fn = nn.CrossEntropyLoss(label_smoothing=0.1)
        print("✅ Using standard CrossEntropyLoss")
    return loss_fn

def create_optimizer_scheduler(model, config):
    """Create optimizer and scheduler"""
    optimizer = torch.optim.AdamW(
        model.parameters(), lr=config.LEARNING_RATE, weight_decay=config.WEIGHT_DECAY
    )
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=10, T_mult=2
    )
    return optimizer, scheduler

# Initialize model components
model = create_model(Config)
loss_fn = create_loss_function(Config)
optimizer, scheduler = create_optimizer_scheduler(model, Config)

print(f"🤖 Model initialized:")
print(f"   - Parameters: {sum(p.numel() for p in model.parameters()):,}")


✅ Using weighted CrossEntropyLoss
🤖 Model initialized:
   - Parameters: 4,029,325


In [7]:
# Training Functions
def train_one_epoch(loader, model, optimizer, loss_fn, device):
    """Train for one epoch"""
    model.train()
    total_loss = 0
    all_preds, all_targets = [], []

    pbar = tqdm(loader, desc="🏃 Training")
    for images, targets in pbar:
        images, targets = images.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        all_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
        all_targets.extend(targets.detach().cpu().numpy())
        
        pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='macro')
    
    return {'loss': avg_loss, 'accuracy': accuracy, 'f1': f1}

def validate_one_epoch(loader, model, loss_fn, device):
    """Validate for one epoch"""
    model.eval()
    total_loss = 0
    all_preds, all_targets = [], []

    with torch.no_grad():
        pbar = tqdm(loader, desc="🔍 Validation", leave=False)
        for images, targets in pbar:
            images, targets = images.to(device), targets.to(device)
            
            outputs = model(images)
            loss = loss_fn(outputs, targets)

            total_loss += loss.item()
            all_preds.extend(outputs.argmax(dim=1).detach().cpu().numpy())
            all_targets.extend(targets.detach().cpu().numpy())
            
            pbar.set_postfix({'Loss': f'{loss.item():.4f}'})

    avg_loss = total_loss / len(loader)
    accuracy = accuracy_score(all_targets, all_preds)
    f1 = f1_score(all_targets, all_preds, average='macro', zero_division=0)
    
    return {'loss': avg_loss, 'accuracy': accuracy, 'f1': f1}

print("✅ Training functions defined")


✅ Training functions defined


In [8]:
# Initialize Wandb
wandb.init(
    project="computervisioncompetition-cv3",
    name=f'{datetime.datetime.now().strftime("%Y%m%d")}_{Config.MODEL_NAME}_clean_pipeline',
    config={
        "model": Config.MODEL_NAME,
        "epochs": Config.EPOCHS,
        "batch_size": Config.BATCH_SIZE,
        "learning_rate": Config.LEARNING_RATE,
        "img_size": Config.IMG_SIZE,
        "use_class_weights": Config.USE_CLASS_WEIGHTS,
        "use_class_specific_aug": Config.USE_CLASS_SPECIFIC_AUG,
        "train_samples": len(train_df),
        "val_samples": len(val_df),
    }
)

print("✅ Wandb initialized")


wandb: Currently logged in as: visioncraft_james (visioncraft_james-open-university-of-korea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


✅ Wandb initialized


In [ ]:
# 개선된 Training Loop (오버피팅 방지)
best_val_f1 = 0
patience_counter = 0

print("🚀 Starting improved training...")
print(f"   - Total epochs: {Config.EPOCHS}")
print(f"   - Early stopping patience: {Config.EARLY_STOPPING_PATIENCE}")
print(f"   - Using class weights: {Config.USE_CLASS_WEIGHTS}")
print(f"   - Validation-based early stopping: ✅")
print("=" * 70)

for epoch in range(Config.EPOCHS):
    # Training phase
    train_metrics = train_one_epoch(train_loader, model, optimizer, loss_fn, Config.DEVICE)
    
    # Validation phase (핵심!)
    val_metrics = validate_one_epoch(val_loader, model, loss_fn, Config.DEVICE)
    
    # Update scheduler
    scheduler.step()
    current_lr = scheduler.get_last_lr()[0]
    
    # 오버피팅 감지
    train_val_gap = train_metrics['f1'] - val_metrics['f1']
    
    # Early stopping logic (Validation F1 기준)
    log_msg = f"Epoch {epoch+1:2d}/{Config.EPOCHS} | "
    log_msg += f"Train F1: {train_metrics['f1']:.4f} | "
    log_msg += f"Val F1: {val_metrics['f1']:.4f} | "
    log_msg += f"Gap: {train_val_gap:+.4f} | "
    log_msg += f"LR: {current_lr:.6f}"
    
    # 오버피팅 경고
    if train_val_gap > 0.05:
        log_msg += " | ⚠️ Overfitting!"
    
    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        patience_counter = 0
        torch.save(model.state_dict(), f'best_val_model_f1_{best_val_f1:.4f}.pth')
        log_msg += " | 💾 Best Val!"
    else:
        patience_counter += 1
        if patience_counter >= Config.EARLY_STOPPING_PATIENCE:
            print(f"🛑 Early stopping! Best Val F1: {best_val_f1:.4f}")
            break
    
    # Log to wandb
    wandb.log({
        "epoch": epoch,
        "train_loss": train_metrics['loss'],
        "train_acc": train_metrics['accuracy'],
        "train_f1": train_metrics['f1'],
        "val_loss": val_metrics['loss'],
        "val_acc": val_metrics['accuracy'],
        "val_f1": val_metrics['f1'],
        "train_val_gap": train_val_gap,
        "learning_rate": current_lr,
        "best_val_f1": best_val_f1
    })
    
    print(log_msg)

print(f"✅ Training completed! Best Validation F1: {best_val_f1:.4f}")
print(f"📊 Final Train-Val Gap: {train_metrics['f1'] - val_metrics['f1']:+.4f}")

# 오버피팅 진단
if best_val_f1 < 0.85:
    print("⚠️ 낮은 Validation 성능 - 모델 개선 필요")
elif train_metrics['f1'] - val_metrics['f1'] > 0.05:
    print("⚠️ 심각한 오버피팅 감지 - 정규화 강화 필요")


🚀 Starting training...
   - Total epochs: 50
   - Early stopping patience: 5
   - Using class weights: True


🏃 Training: 100%|██████████| 78/78 [00:17<00:00,  4.35it/s, Loss=1.5012]


Epoch  1/50 | Train F1: 0.3514 | Val F1: 0.7086 | LR: 0.000098 | 💾 Best!


🏃 Training: 100%|██████████| 78/78 [00:08<00:00,  9.33it/s, Loss=1.3025]


Epoch  2/50 | Train F1: 0.6530 | Val F1: 0.7977 | LR: 0.000090 | 💾 Best!


...

Epoch 28/50 | Train F1: 0.9716 | Val F1: 0.9350 | LR: 0.000002


🏃 Training: 100%|██████████| 78/78 [00:08<00:00,  8.98it/s, Loss=0.8226]


Epoch 29/50 | Train F1: 0.9645 | Val F1: 0.9289 | LR: 0.000001


🏃 Training: 100%|██████████| 78/78 [00:08<00:00,  9.02it/s, Loss=0.7389]
                                                                           

🛑 Early stopping! Best Val F1: 0.9385
✅ Training completed! Best Validation F1: 0.9385


## 4. Inference & Submission


In [10]:
# Inference Functions
def load_best_model(model, best_f1):
    """Load the best saved model"""
    try:
        model_path = f'best_model_f1_{best_f1:.4f}.pth'
        model.load_state_dict(torch.load(model_path))
        print(f"✅ Loaded best model: {model_path}")
    except:
        print("⚠️ Could not load best model, using current model")
    return model

def predict_with_tta(model, test_loader, device, use_tta=True):
    """Make predictions with optional TTA"""
    model.eval()
    all_predictions = []
    
    # Base predictions
    print("🔮 Base inference...")
    base_preds = []
    with torch.no_grad():
        for images, _ in tqdm(test_loader, desc="Base inference"):
            images = images.to(device)
            outputs = model(images)
            probs = torch.softmax(outputs, dim=1)
            base_preds.extend(probs.detach().cpu().numpy())
    
    all_predictions.append(np.array(base_preds))
    
    # TTA predictions
    if use_tta:
        print("🔮 TTA inference...")
        tta_transforms = get_tta_transforms(Config.IMG_SIZE)
        
        for i, tta_transform in enumerate(tta_transforms[1:], 1):
            # Create TTA dataset
            tta_dataset = DocumentDataset(
                os.path.join(Config.DATA_PATH, "sample_submission.csv"),
                os.path.join(Config.DATA_PATH, "test"),
                tta_transform, is_train=False
            )
            tta_loader = DataLoader(
                tta_dataset, batch_size=Config.BATCH_SIZE, shuffle=False,
                num_workers=Config.NUM_WORKERS, pin_memory=True
            )
            
            tta_preds = []
            with torch.no_grad():
                for images, _ in tqdm(tta_loader, desc=f"TTA {i}"):
                    images = images.to(device)
                    outputs = model(images)
                    probs = torch.softmax(outputs, dim=1)
                    tta_preds.extend(probs.detach().cpu().numpy())
            
            all_predictions.append(np.array(tta_preds))
    
    # Average predictions
    if len(all_predictions) > 1:
        print(f"📊 Averaging {len(all_predictions)} predictions...")
        final_preds = np.mean(all_predictions, axis=0)
    else:
        final_preds = all_predictions[0]
    
    return np.argmax(final_preds, axis=1)

def create_submission(predictions, best_f1, use_tta=True):
    """Create submission file"""
    # Load sample submission
    sample_sub = pd.read_csv(os.path.join(Config.DATA_PATH, "sample_submission.csv"))
    submission = sample_sub.copy()
    submission['target'] = predictions
    
    # Generate filename
    today = datetime.datetime.now().strftime("%Y%m%d")
    tta_suffix = "_TTA" if use_tta else ""
    cw_suffix = "_CW" if Config.USE_CLASS_WEIGHTS else ""
    ca_suffix = "_CA" if Config.USE_CLASS_SPECIFIC_AUG else ""
    
    filename = f"submission/sub_{today}_{Config.MODEL_NAME}_f1_{best_f1:.4f}{tta_suffix}{cw_suffix}{ca_suffix}.csv"
    
    # Save submission
    os.makedirs("submission", exist_ok=True)
    submission.to_csv(filename, index=False)
    
    return filename, submission

print("✅ Inference functions defined")


✅ Inference functions defined


In [11]:
# Run Inference
model = load_best_model(model, best_val_f1)

# Make predictions with TTA
predictions = predict_with_tta(model, test_loader, Config.DEVICE, use_tta=True)

# Create submission file
filename, submission = create_submission(predictions, best_val_f1, use_tta=True)

print(f"\n🎯 Inference completed!")
print(f"   - Predictions: {len(predictions):,} samples")
print(f"   - Best Validation F1: {best_val_f1:.4f}")
print(f"   - Submission saved: {filename}")

print(f"\n📊 Submission preview:")
print(submission.head())

print(f"\n🎉 Clean pipeline completed successfully!")
print(f"\n📈 Key Features Applied:")
print(f"   ✅ EDA-based class weights for imbalance")
print(f"   ✅ Train/Test domain adaptation augmentations")
print(f"   ✅ Class-specific preprocessing")
print(f"   ✅ Proper train/validation split")
print(f"   ✅ Early stopping on validation F1")
print(f"   ✅ Test Time Augmentation")
print(f"   ✅ Clean, maintainable code structure")


✅ Loaded best model: best_model_f1_0.9385.pth
🔮 Base inference...


Base inference: 100%|██████████| 197/197 [00:13<00:00, 14.15it/s]


🔮 TTA inference...


TTA 1: 100%|██████████| 197/197 [00:05<00:00, 33.35it/s]

📊 Averaging 2 predictions...

🎯 Inference completed!
   - Predictions: 3,140 samples
   - Best Validation F1: 0.9385
   - Submission saved: submission/sub_20250908_efficientnet_b0_f1_0.9385_TTA_CW_CA.csv

📊 Submission preview:
                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg       1
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg       3
4  00901f504008d884.jpg       2

🎉 Clean pipeline completed successfully!

📈 Key Features Applied:
   ✅ EDA-based class weights for imbalance
   ✅ Train/Test domain adaptation augmentations
   ✅ Class-specific preprocessing
   ✅ Proper train/validation split
   ✅ Early stopping on validation F1
   ✅ Test Time Augmentation
   ✅ Clean, maintainable code structure
